# **Interfaccia I2C - LPS331AP**

Il notebook mostra come usare $\texttt{Analog Discovery 2}$ per instaurare una connessione seriale di tipo $\texttt{I2C}$.

In questo caso lo script comunica con il sensore $\texttt{LPS331AP}$ e utilizza i seguenti registri:

* `0x20` registro di configurazione generale per accendere (!) e impostare i parametri di misura
* `0x2B` registro di lettura del LSB della Temperatura 

**Interpretazione della lettura**. Il sensore risponde con 2 bytes che codificano la temperatura. I due bytes vengono entrambe codificati come un `uint16` che chiameremo `utemp`. La conversione finale richiede

        temp = 42.5 + utemp/480

**Lettura della pressione**. può essere implementata in modo analogo, cercando il registro giusto e la conversione corretta

**Qualche accortezza**. Ricordiamo che il sensore fornisce i necessari *pull-up* per il corretto funzionamento dell'interfaccia I2C, quindi è fondamentale che il sensore sia alimentato altrimenti l'avvio del bus I2C darà un errore.

In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time

# -[LPS331AP - Qualche registro]---------------------------------------------------
WHO_AM_I = 0x0F
CTRL_REG1 = 0x20
STATUS_REG = 0x27
TEMP_OUT_L = 0x2B
TEMP_OUT_H = 0x2C
SAD = 0x5d

# -[Configurazione AD2]---------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
ad2.vdd = 3.3
ad2.power(True)
#   3. Impostazione bus I2C
i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
#   4. Apre la comunicazione I2C come Master
sht = tdwf.I2Cdevice(ad2.hdwf,SAD)

# -[Configurazione sensore]------------------------------------------------------
sht.write([CTRL_REG1,0x00]) # Seleziona registro

sht.write([CTRL_REG1,0x90]) # Seleziona registro


# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_key(event):
    global flag_run
    if event.key == 'escape':  # termina programma
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, ax = plt.subplots(figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect("key_press_event", on_key)

#   2. Inizializzazione variabili
TTv = []
mytime = []
flag_first = True
flag_run = True

start_time = time.time()

#   3. Ciclo di misura
while flag_run:
    
    time.sleep(0.2)
    sht.write(TEMP_OUT_L+0x80)  # scrive un registro (con autoincremento)
    sht.read(2) # legge 2 bytes
    

    # Complemento a due
    TT = sht.vals[0] + 256*sht.vals[1]
    if TT >= 0x8000:
        TT = TT-0x10000
    rTT = 42.5+TT/480 # Conversione temperatura
    TTv.append(rTT)
    print(f"T = {rTT:.2f}C")

    mytime.append(time.time()-start_time)
    if flag_first:
        flag_first = False
        hp1, = plt.plot(mytime, TTv, "o", color="tab:orange")
        plt.grid(True)
        plt.ylabel("Temperature [C]")
        plt.xlabel("Time [s]")
        plt.title("Press ESC to exit")
        plt.show(block=False)    
        plt.tight_layout()
        
    else:
        hp1.set_ydata(TTv)
        hp1.set_xdata(mytime)
        ax.relim()           
        ax.autoscale_view() 
        fig.canvas.draw()
        fig.canvas.flush_events() 
# ---------------------------------------

plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321B19D0C, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
T = 21.31C
T = 21.31C
T = 21.30C
T = 21.30C
T = 21.30C
T = 21.30C
T = 21.32C
T = 21.32C
T = 21.32C
T = 21.28C
T = 21.28C
T = 21.28C
T = 21.28C
T = 21.30C
T = 21.30C
T = 21.30C


KeyboardInterrupt: 